In [96]:
import numpy as np
import pandas as pd
import random
import itertools 
import os
from collections import defaultdict
import json
import random
import numpy as np
from time import time
from tqdm import tqdm

In [182]:
#creating smaller simulations .npy file
red = ['0'] * 26
black = ['1'] * 26
deck = black + red

#set seed
np.random.seed(123)

#shuffle deck for num iterations (10000 in this case)
results = []
for i in tqdm(range(10000)):
    np.random.shuffle(deck)
    results += [int(''.join(deck),2)]

np.save('results_10000', results)

100%|██████████| 10000/10000 [00:00<00:00, 350547.76it/s]


In [159]:
#load in list of simulations, convert ot binary
def load_process_simulations(file_path): 
    simulations = np.load(file_path)
    
    def int_to_bit(n):
        binary = bin(n)[2:] #what? 
        return binary.zfill(52)
    
    binary_simulations = [int_to_bit(sim) for sim in simulations]             
    return binary_simulations

In [181]:
#for testing
processed_sims = load_process_simulations('results_10000.npy')

In [174]:
processed_sims_total = load_process_simulations('results.npy')

In [175]:
def variation1(deck, player1_sequence, player2_sequence):
    player1_cards = 0
    player2_cards = 0
    pile = 0
    
    for i in range(len(deck) - 2):
        current_sequence = deck[i:i+3]
        pile += 1
        if current_sequence == player1_sequence:
            player1_cards += pile
            pile = 0
        elif current_sequence == player2_sequence:
            player2_cards += pile
            pile = 0 
    return player1_cards, player2_cards

In [176]:
def variation2(deck, player1_sequence, player2_sequence):
    player1_tricks = 0
    player2_tricks = 0
    
    for i in range(len(deck) - 2):
        current_sequence = deck[i:i+3]
        if current_sequence == player1_sequence:
            player1_tricks += 1
        elif current_sequence == player2_sequence:
            player2_tricks += 1
    return player1_tricks, player2_tricks

In [177]:
def analyze_all_combinations_v2(processed_sims):
    all_sequences = [''.join(seq) for seq in itertools.product('01', repeat=3)]
    
    results_v1 = []
    results_v2 = []

    for p1 in all_sequences:
        for p2 in all_sequences:
            if p1 != p2:

                p1_wins_v1, p2_wins_v1 = 0,0
                p1_wins_v2, p2_wins_v2 = 0,0

                for game in processed_sims:
                    p1_cards_v1, p2_cards_v1 = variation1(game, p1, p2)
                    if p1_cards_v1 > p2_cards_v1:
                        p1_wins_v1 += 1
                    elif p2_cards_v1 > p1_cards_v1:
                        p2_wins_v1 += 1
                
                    p1_cards_v2, p2_cards_v2 = variation2(game, p1, p2)
                    if p1_cards_v2 > p2_cards_v2:
                        p1_wins_v2 += 1
                    elif p2_cards_v2 > p1_cards_v2:
                        p2_wins_v2 += 1
                
                results_v1.append({'Sequence 1': p1, 'Sequence 2': p2, 
                                   'Player 1 Wins': p1_wins_v1, 'Player 2 Wins': p2_wins_v1, 
                                   'Player 1 Win %': round((p1_wins_v1/len(processed_sims))*100,2),
                                   'Player 2 Win %': round((p2_wins_v1/len(processed_sims))*100,2)})
                results_v2.append({'Sequence 1': p1, 'Sequence 2': p2, 
                                   'Player 1 Wins': p1_wins_v2, 'Player 2 Wins': p2_wins_v2, 
                                   'Player 1 Win %': round((p1_wins_v2/len(processed_sims))*100,2),
                                   'Player 2 Win %': round((p2_wins_v2/len(processed_sims))*100,2)})
   
    aggreg_df1 = pd.DataFrame(results_v1)
    aggreg_df2 = pd.DataFrame(results_v2)

    return aggreg_df1, aggreg_df2

In [178]:
aggreg_df1, aggreg_df2 = analyze_all_combinations_v2(processed_sims_total) 

In [179]:
aggreg_df1.head()

,Sequence 1,Sequence 2,Player 1 Wins,Player 2 Wins,Player 1 Win %,Player 2 Win %
0,000,001,475278,501157,47.53,50.12
1,000,010,284778,697059,28.48,69.71
2,000,011,194577,780437,19.46,78.04
3,000,100,40,999931,0.00,99.99
4,000,101,264900,711318,26.49,71.13


In [180]:
aggreg_df2.head()

,Sequence 1,Sequence 2,Player 1 Wins,Player 2 Wins,Player 1 Win %,Player 2 Win %
0,000,001,361603,510470,36.16,51.05
1,000,010,401158,499479,40.12,49.95
2,000,011,330943,519062,33.09,51.91
3,000,100,361242,510480,36.12,51.05
4,000,101,376888,485568,37.69,48.56


In [ ]:
#10000: P1 47.89, P2 49.87; P1 36.64, P2 50.90
#20000: P1 47.70, P2 50.02; P1 36.11, P2 51.40
#40000: P1 47.88, P2 49.86; P1 36.38, P2 50.81
#80000: P1 47.29, P2 50.40; P1 36.13, P2 51.06
#160000: P1 47.38, P2 50.33; P1 36.25, P2 51.03

#totals for actual sim (results.npy)
#       P1 47.53, P2 50.12; P1 36.16, P2 51.05

Everything below this is redundant/for testing

---

In [164]:
# def analyze_all_combinations(processed_sims):
#     all_sequences = [''.join(seq) for seq in itertools.product('01', repeat=3)]
#     results1 = {(p1, p2): {'player1_wins': 0, 'player2_wins': 0}
#         for p1 in all_sequences for p2 in all_sequences if p1 != p2}
#     results2 = {(p1, p2): {'player1_wins': 0, 'player2_wins': 0}
#         for p1 in all_sequences for p2 in all_sequences if p1 != p2}
    
#     for game in processed_sims:
#         for (p1_seq, p2_seq), result in results1.items():
#             p1_cards, p2_cards = variation1(game, p1_seq, p2_seq)
#             if p1_cards > p2_cards:
#                 result['player1_wins'] += 1
#             elif p2_cards > p1_cards:
#                 result['player2_wins'] += 1

#     for game in processed_sims:
#         for (p1_seq, p2_seq), result in results2.items():
#             p1_tricks, p2_tricks = variation2(game, p1_seq, p2_seq)
#             if p1_tricks > p2_tricks:
#                 result['player1_wins'] += 1
#             elif p2_cards > p1_cards:
#                 result['player2_wins'] += 1
#     return results1, results2

In [165]:
# results1, results2 = analyze_all_combinations(processed_sims)

In [166]:
# df1 = pd.DataFrame.from_dict(results1, orient = 'index')
# df1.reset_index(inplace=True)
# df1.columns = ['Sequence 1','Sequence 2', 'Player 1 Wins', 'Player 2 Wins']
# df1.head()

In [167]:
# df2 = pd.DataFrame.from_dict(results2, orient = 'index')
# df2.reset_index(inplace=True)
# df2.columns = ['Sequence 1','Sequence 2', 'Player 1 Wins', 'Player 2 Wins']
# df2.head()

In [168]:
# df1.to_csv('Variation1.csv', index=False)

# df2.to_csv('Variation2.csv', index=False)